In [ ]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable, Type
import abc

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
from torch import Tensor
import matplotlib.pyplot as plt

from models.ridge_ALOOCV import fit_ridge_ALOOCV
from models.sandwiched_least_squares import sandwiched_LS_dense, sandwiched_LS_diag, sandwiched_LS_scalar

def acc_from_logits(logits, y, n_classes):
    if n_classes==2:
        pred = (logits>0).float()
    else:
        pred = torch.argmax(logits, dim=1)
        y = torch.argmax(y, dim=1)
    acc = (pred==y).float().mean()
    return acc



def generate_concentric_rings(n_rings_per_class=2, n_classes=3, n_samples=2000, device="cpu") -> Tuple[np.ndarray, np.ndarray]:
    X = []
    y = []
    n_rings = n_rings_per_class * n_classes
    samples_per_ring = n_samples // n_rings
    for i in range(n_rings):
        radius = i + 1
        theta = np.linspace(0, 2 * np.pi, samples_per_ring)
        noise_scale = 0.5
        x1 = radius * np.cos(theta) + np.random.rand(samples_per_ring) * noise_scale
        x2 = radius * np.sin(theta) + np.random.rand(samples_per_ring) * noise_scale
        X.append(np.vstack((x1, x2)).T)
        y.append(np.full(samples_per_ring, i % n_classes))
    X = np.vstack(X)
    y = np.hstack(y)
    perm = np.random.permutation(len(X))
    X = X[perm] / np.max(X)
    y = y[perm]

    #plot
    plt.figure(figsize=(8, 8))
    for i in range(n_classes):
        plt.scatter(X[y == i][:, 0], X[y == i][:, 1], label=f'Class {i}')
    plt.legend()
    plt.title('Concentric Rings Dataset')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()

    y = torch.tensor(y).float()
    X = torch.tensor(X).float()
    if n_classes > 2:
        y = torch.nn.functional.one_hot(y.to(torch.int64), n_classes).float()
    else:
        y = y.unsqueeze(1)

    r = int(0.5 * len(X))
    X_train, y_train, X_test, y_test = X[:r], y[:r], X[r:], y[r:]
    return (X_train.to(device),
            y_train.to(device), 
            X_test.to(device), 
            y_test.to(device) )




# Generate dataset
# n_classes = 2
# D = 10
# X_train, y_train, X_test, y_test = make_spirals_dataset_Rd(
#     n_samples=1000, n_classes=n_classes, noise=0.1, D=D, train_test_ratio=0.8
#     )
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
n_classes = 3
D = 2
device = "cpu"
X_train, y_train, X_test, y_test = generate_concentric_rings(
    n_rings_per_class=2, n_classes=n_classes, n_samples=2000, device=device
    )
if n_classes > 2:
    loss_fn = nn.functional.cross_entropy
else:
    loss_fn = nn.functional.binary_cross_entropy_with_logits

In [ ]:
from models.base import LogisticRegression
    
model = LogisticRegression(
    n_classes, l2_lambda=0.1, max_iter=100
     )

results = []
for i in range(5):
    model.fit(X_train, y_train)
    out_train = model(X_train)
    out_test = model(X_test)
    ce_train = loss_fn(out_train, y_train)
    ce_test = loss_fn(out_test, y_test)
    acc_train = acc_from_logits(out_train, y_train, n_classes)
    acc_test = acc_from_logits(out_test, y_test, n_classes)
    results.append(torch.tensor([ce_train, ce_test, acc_train, acc_test]))
results = torch.stack(results)
print("train ce", results[:, 0].mean(), "std", results[:, 0].std())
print("test ce", results[:, 1].mean(), "std", results[:, 1].std())
print("train acc", results[:, 2].mean(), "std", results[:, 2].std())
print("test acc", results[:, 3].mean(), "std", results[:, 3].std())

In [ ]:
from models.random_feature_representation_boosting import GradientRFRBoostClassifier

seed=0
np.random.seed(seed)
torch.manual_seed(seed)

model = GradientRFRBoostClassifier(
        in_dim=D,
        n_classes=n_classes,
        hidden_dim=2,
        n_layers=4,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_cls=0.001,
        l2_ghat=0.00001,
        boost_lr=1.0,
        feature_type="SWIM",
        upscale_type="identity",
        lbfgs_lr=1.0,
        lbfgs_max_iter=300,
    )

results = []
for i in range(2):
    model.fit(X_train, y_train)
    out_train = model(X_train)
    out_test = model(X_test)
    ce_train = loss_fn(out_train, y_train)
    ce_test = loss_fn(out_test, y_test)
    acc_train = acc_from_logits(out_train, y_train, n_classes)
    acc_test = acc_from_logits(out_test, y_test, n_classes)
    results.append(torch.tensor([ce_train, ce_test, acc_train, acc_test]))
results = torch.stack(results)
print("train ce", results[:, 0].mean(), "std", results[:, 0].std())
print("test ce", results[:, 1].mean(), "std", results[:, 1].std())
print("train acc", results[:, 2].mean(), "std", results[:, 2].std())
print("test acc", results[:, 3].mean(), "std", results[:, 3].std())

In [ ]:
def plot_features(X_train, y_train, X_test, y_test):
        plt.figure(figsize=(12, 6))

        if y_train.size(1) == 1:
            y_train = torch.nn.functional.one_hot(y_train[:, 0].to(torch.int64), 2).float()
            y_test = torch.nn.functional.one_hot(y_test[:, 0].to(torch.int64), 2).float()

        X_train = X_train.cpu().numpy()
        y_train = y_train.cpu().numpy()
        X_test = X_test.cpu().numpy()
        y_test = y_test.cpu().numpy()


        plt.subplot(1, 2, 1)
        for i in range(n_classes):
            plt.scatter(X_train[y_train[:, i] == 1][:, 0], X_train[y_train[:, i] == 1][:, 1], label=f'Class {i}')
        plt.title('Training Data')
        plt.xlabel('x1')
        plt.ylabel('x2')
        plt.legend()

        plt.subplot(1, 2, 2)
        for i in range(n_classes):
            plt.scatter(X_test[y_test[:, i] == 1][:, 0], X_test[y_test[:, i] == 1][:, 1], label=f'Class {i}')
        plt.title('Test Data')
        plt.xlabel('x1')
        plt.ylabel('x2')
        plt.legend()

        plt.show()


def see_results_for_every_layer(X_train, y_train, X_test, y_test, model, loss_fn):
    with torch.no_grad():
        X0_train = X_train
        X0_test = X_test

        X_train = model.upscale(X0_train)
        X_test = model.upscale(X0_test)

        pred_train = model.top_level_modules[0](X_train)
        pred_test = model.top_level_modules[0](X_test)

        ce_train = loss_fn(pred_train, y_train)
        ce_test = loss_fn(pred_test, y_test)
        acc_train = acc_from_logits(pred_train, y_train, n_classes)
        acc_test = acc_from_logits(pred_test, y_test, n_classes)
        print(f"Train ce at layer 0: {ce_train}")
        print(f"Test ce at layer 0: {ce_test}")
        print(f"Train acc at layer 0: {acc_train}")
        print(f"Test acc at layer 0: {acc_test}")
        print()
        plot_features(X_train, y_train, X_test, y_test)
        
        for t, (feat_layer, ghat_layer, classifier) in enumerate(zip(model.random_feature_layers, 
                                                                     model.ghat_boosting_layers, 
                                                                     model.top_level_modules[1:])):
            features_train = feat_layer(X_train, X0_train)
            features_test = feat_layer(X_test, X0_test)
            X_train = X_train + model.boost_lr * ghat_layer(features_train)
            X_test = X_test + model.boost_lr * ghat_layer(features_test)
            
            pred_train = classifier(X_train)
            pred_test = classifier(X_test)

            ce_train = loss_fn(pred_train, y_train)
            ce_test = loss_fn(pred_test, y_test)
            acc_train = acc_from_logits(pred_train, y_train, n_classes)
            acc_test = acc_from_logits(pred_test, y_test, n_classes)

            print(f"Train ce at layer {t+1}: {ce_train}")
            print(f"Test ce at layer {t+1}: {ce_test}")
            print(f"Train acc at layer {t+1}: {acc_train}")
            print(f"Test acc at layer {t+1}: {acc_test}")
            print()
            plot_features(X_train, y_train, X_test, y_test)

see_results_for_every_layer(X_train, y_train, X_test, y_test, model, loss_fn)


# End2End

In [ ]:
from models.end2end import End2EndMLPResNet

n_blocks = 5
model = End2EndMLPResNet(
        in_dim=2,
        hidden_dim=2,
        bottleneck_dim=128,
        out_dim= (n_classes if n_classes>2 else 1),
        n_blocks=n_blocks,
        loss = ("cce" if n_classes>2 else "bce"),
        lr = 0.1,
        n_epochs = 30,
        end_lr_factor= 0.01,
        weight_decay = 0.00001,
        batch_size = 64,
        upsample = False,
        activation = nn.Tanh(),
        )

results = []
for i in range(1):
    model.fit(X_train, y_train)
    model.eval()
    out_train = model(X_train)
    out_test = model(X_test)
    ce_train = loss_fn(out_train, y_train)
    ce_test = loss_fn(out_test, y_test)
    acc_train = acc_from_logits(out_train, y_train, n_classes)
    acc_test = acc_from_logits(out_test, y_test, n_classes)
    results.append(torch.tensor([ce_train, ce_test, acc_train, acc_test]))
results = torch.stack(results)
print("train ce", results[:, 0].mean(), "std", results[:, 0].std())
print("test ce", results[:, 1].mean(), "std", results[:, 1].std())
print("train acc", results[:, 2].mean(), "std", results[:, 2].std())
print("test acc", results[:, 3].mean(), "std", results[:, 3].std())

In [ ]:
def plot_resnet(X_train, y_train, X_test, y_test, model):
    with torch.no_grad():
        # for each layer, calculate the activations, and plot
        X_train = model.upsample(X_train)
        X_test = model.upsample(X_test)
        plot_features(X_train, y_train, X_test, y_test)

        for t in range(n_blocks):
            X_train = X_train + model.residual_blocks[t](X_train)
            X_test = X_test + model.residual_blocks[t](X_test)
            plot_features(X_train, y_train, X_test, y_test)
plot_resnet(X_train, y_train, X_test, y_test, model)

# How do i want to set up the experiments?

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from models.base import FittableModule

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

class SKLearnWrapper(BaseEstimator):
    def __init__(self, modelClass=None, **model_params):
        self.modelClass = modelClass
        self.model_params = model_params
        self.model = None

    def fit(self, X, y):
        self.model = self.modelClass(**self.model_params)
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def set_params(self, **params):
        self.modelClass = params.pop('modelClass', self.modelClass)
        self.model_params.update(params)
        return self

    def get_params(self, deep=True):
        params = {'modelClass': self.modelClass}
        params.update(self.model_params)
        return params
    
    def score(self, X, y):
        logits = self.model(X)
        if y.size(1) == 1:
            pred = (logits>0).float()
        else:
            pred = torch.argmax(logits, dim=1)
            y = torch.argmax(y, dim=1)
        acc = (pred==y).float().mean()
        return acc.detach().cpu().item()
    

import matplotlib.patches as patches

def save_feature_plot(
        X_train, 
        y_train, 
        X_test, 
        y_test, 
        model, 
        n_classes, 
        file_name: str,
        save_dir: str = "save/ConcentricCircles/"
        ):
    """
    Saves train and test features side by side for visualization at a given layer.
    
    Args:
        X_train (torch.Tensor): Features of the training set (shape: [n_samples, 2]).
        y_train (torch.Tensor): Labels of the training set (one-hot or integer-encoded).
        X_test (torch.Tensor): Features of the test set (shape: [n_samples, 2]).
        y_test (torch.Tensor): Labels of the test set (one-hot or integer-encoded).
        n_classes (int): Number of classes in the dataset.
        file_name (str): Name of the file to save the plot.
        save_dir (str): Directory to save the plot.
    """
    # Obtain the features at each layer
    train_features = [X_train]
    test_features = [X_test]
    if type(model) == GradientRFRBoostClassifier:
        X0_train = X_train
        X0_test = X_test
        for t, (feat_layer, ghat_layer, classifier) in enumerate(zip(model.random_feature_layers, 
                                                                     model.ghat_boosting_layers, 
                                                                     model.top_level_modules[1:])):
            features_train = feat_layer(X_train, X0_train)
            features_test = feat_layer(X_test, X0_test)
            X_train = X_train + model.boost_lr * ghat_layer(features_train)
            X_test = X_test + model.boost_lr * ghat_layer(features_test)
            train_features.append(X_train)
            test_features.append(X_test)
    elif type(model) == End2EndMLPResNet:
        for resblock in model.residual_blocks:
            X_train = X_train + resblock(X_train)
            X_test = X_test + resblock(X_test)
            train_features.append(X_train)
            test_features.append(X_test)
    else:
        raise ValueError("Model type not recognized.")
    
    # Convert labels to integers if they're one-hot encoded
    train_labels_np = y_train.argmax(dim=1).cpu().numpy() if n_classes > 2 else y_train.cpu().numpy()
    test_labels_np = y_test.argmax(dim=1).cpu().numpy() if n_classes > 1 else y_test.cpu().numpy()


    # Plot the features
    T = len(train_features)
    for name_train_or_test, features, labels in [("train", train_features, train_labels_np), 
                                                 ("test", test_features, test_labels_np)]:
        fig, axes = plt.subplots(1, T, figsize=(T * 4, 4), dpi=100)
        for t in range(T):
            ax = axes[t]
            feat = features[t].detach().cpu().numpy()
            for j in range(feat.shape[1]):
                feat[:, j] = (feat[:, j] - feat[:, j].min()) / (feat[:, j].max() - feat[:, j].min())
            for i in range(n_classes):
                ax.scatter(feat[labels == i][:, 0], feat[labels == i][:, 1])

            # Remove titles and adjust axis
            ax.axis('equal')
            ax.axis('off')

            # Add black border as a rectangle
            rect = patches.Rectangle(
                (0, 0), 1, 1, transform=ax.transAxes,  # Set size relative to axes
                linewidth=2, edgecolor='black', facecolor='none', zorder=10
            )
            ax.add_patch(rect)

        # Add vertical label for the model name
        name = "MLP ResNet" if type(model) == End2EndMLPResNet else "GRFRBoost (ours)"
        if "GRFRBoost" in name:
            if model.randfeat_x0_dim == 0:
                name += " x0 only"
            if model.feature_type == "SWIM":
                name += " SWIM"
        fig.text(0.02, 0.5, name, fontsize=18, rotation='vertical', va='center', ha='center')

        plt.tight_layout()
        plt.subplots_adjust(left=0.03)  # Add space for the model name
        plt.savefig(f"{save_dir}/{file_name}_{name_train_or_test}.png", bbox_inches='tight', dpi=300)
        plt.close(fig)




def run_concentric_rings_experiment(
        seed=0,
        k_folds=5,
        n_classes=3,
        n_rings_per_class=2,
        n_samples=2000,
        device="cuda",
        ):
    """Runs the concentric circle experiments and saves the plots and results."""
    # Set seed
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    # Create dataset
    X_train, y_train, X_test, y_test = generate_concentric_rings(
        n_rings_per_class, n_classes, n_samples, device
    )

    # Define models and their hyperparameter grids
    n_layers = 4 # TODO CHANGE
    model_specs = {
        "GradientRFRBoost": {
            'modelClass': [GradientRFRBoostClassifier],
            'l2_cls': np.logspace(-7, -1, 7),
            'l2_ghat': np.logspace(-7, -1, 7),
            'in_dim': [2],
            'n_classes': [n_classes],
            'hidden_dim': [2],
            'n_layers': [n_layers],
            'randfeat_xt_dim': [128],
            'randfeat_x0_dim': [128],
            'feature_type': ["iid"],
            'upscale_type': ["identity"],
        },
        "GradientRFRBoost xt only": {
            'modelClass': [GradientRFRBoostClassifier],
            'l2_cls': np.logspace(-7, -1, 7),
            'l2_ghat': np.logspace(-7, -1, 7),
            'in_dim': [2],
            'n_classes': [n_classes],
            'hidden_dim': [2],
            'n_layers': [n_layers],
            'randfeat_xt_dim': [128],
            'randfeat_x0_dim': [0],
            'feature_type': ["iid"],
            'upscale_type': ["identity"],
        },
        "GradientRFRBoost SWIM": {
            'modelClass': [GradientRFRBoostClassifier],
            'l2_cls': np.logspace(-7, -1, 7),
            'l2_ghat': np.logspace(-7, -1, 7),
            'in_dim': [2],
            'n_classes': [n_classes],
            'hidden_dim': [2],
            'n_layers': [n_layers],
            'randfeat_xt_dim': [128],
            'randfeat_x0_dim': [128],
            'feature_type': ["SWIM"],
            'upscale_type': ["identity"],
        },
        "GradientRFRBoost xt only SWIM": {
            'modelClass': [GradientRFRBoostClassifier],
            'l2_cls': np.logspace(-7, -1, 7),
            'l2_ghat': np.logspace(-7, -1, 7),
            'in_dim': [2],
            'n_classes': [n_classes],
            'hidden_dim': [2],
            'n_layers': [n_layers],
            'randfeat_xt_dim': [128],
            'randfeat_x0_dim': [0],
            'feature_type': ["SWIM"],
            'upscale_type': ["identity"],
        },
        "Logistic Regression": {
            'modelClass': [LogisticRegression],
            'l2_lambda': np.logspace(-4, -1, 4),
            'n_classes': [n_classes],
        },
        "End2EndMLP": {
            'modelClass': [End2EndMLPResNet],
            'lr': np.logspace(-4, -1, 4),
            'in_dim': [2],
            'hidden_dim': [2],
            'bottleneck_dim': [128],
            'out_dim': [n_classes if n_classes > 2 else 1],
            'n_blocks': [n_layers],
            'loss': ["cce" if n_classes > 2 else "bce"],
            'n_epochs': [30],
            'end_lr_factor': [0.01],
            'weight_decay': [0.00001],
            'batch_size': [64],
            'upsample': [False],
            'activation': [nn.Tanh()],
        },
    }

    # Run experiments
    results = {}
    results_params = {}
    for model_name, param_grid in model_specs.items():
        accuracies = []
        best_params = []
        for i in range(5):
            print(i, model_name)
            # Perform grid search with k-fold cross-validation
            estimator = SKLearnWrapper()
            grid_search = GridSearchCV(
                estimator=estimator,
                param_grid=param_grid,
                cv=k_folds,
            )
            grid_search.fit(X_train, y_train)

            # Evaluate on test set
            best_model = grid_search.best_estimator_
            accuracy = best_model.score(X_test, y_test)
            accuracies.append(accuracy)
            best_params.append(grid_search.best_params_)

            if i == 0 and model_name != "Logistic Regression":
                print("plotting and saving feature evolution point cloud")
                save_feature_plot(X_train, y_train, X_test, y_test, best_model.model, n_classes, model_name)
        results[model_name] = accuracies
        results_params[model_name] = best_params

    return results, results_params

results, results_params = run_concentric_rings_experiment()

for model_name, accs in results.items():
    print(model_name, "acc", np.mean(accs), "std", np.std(accs))
    print(accs)
    print()
for model_name, best_params in results_params.items():
    print(model_name, "best params", best_params)
    print()

# Next: combine the plots into one image for the paper. also combine train vs test plots for appendix

# TODO add batch norm... should help stabilize gradient

# TODO add pointer to prev classifier, for faster training